# Create Test Data for brave-tin-soldier


In [1]:
import pandas as pd
path = "/home/gxu21/ECONET/code/data_process/FairytaleQA_Dataset/split_by_origin/andersen-fairybook/brave-tin-soldier-story/brave-tin-soldier-story.tokens"
df = pd.read_csv(path, sep="	")
#words is a list of all extracted tokens from the .token file
words = list(df["word"])
len(words)
words_dict = dict()
for i,w in enumerate(words):
    words_dict[i] = w

#split the words into a new df of sentences, which includes their start and end token_pos
sentences = []
start = []
end = []
sent = None
for i,w in enumerate(words):
    if sent == None:
        if w == "\t":
            sent+= "'"
        else:
            sent = w
        start_id =i
    else:
        if w == "\t":
            sent+= " '"
        else:
            sent+= " "+ w
    if (w == "." or w == "\t") and (i+1>=len(words) or words[i+1]!='\t'):
        end_id = i
        sentences.append(sent)
        start.append(start_id)
        end.append(end_id)
        sent = None
sent_df = pd.DataFrame({"sentence":sentences})
sent_df["start_id"] = start
sent_df["end_id"] = end
pd.set_option('display.max_colwidth', None)  # or 199
sent_df= sent_df.iloc[5:14]
sent_df

,sentence,start_id,end_id
5,"They were given him for a birthday present , and he stood at the table to set them up .",99,118
6,"The soldiers were all exactly alike , excepting one , who had only one leg ; he had been left to the last , and then there was not enough of the melted tin to finish him , so they made him to stand firmly on one leg , and this caused him to be very remarkable . '",119,177
7,"2,""The table on which the tin soldiers stood , was covered with other playthings , but the most attractive to the eye was a pretty little paper castle .",178,206
8,Through the small windows the rooms could be seen .,207,216
9,"In front of the castle a number of little trees surrounded a piece of looking - glass , which was intended to represent a transparent lake .",217,243
10,"Swans , made of wax , swam on the lake , and were reflected in it .",244,260
11,"All this was very pretty , but the prettiest of all was a tiny little lady , who stood at the open door of the castle ; she , also , was made of paper , and she wore a dress of clear muslin , with a narrow blue ribbon over her shoulders just like a scarf .",261,318
12,"In front of these was fixed a glittering tinsel rose , as large as her whole face . '",319,337
13,"3,""The little lady was a dancer , and she stretched out both her arms , and raised one of her legs so high , that the tin soldier could not see it at all , and he thought that she , like himself , had only one leg . ' '",338,388


In [2]:
df = pd.read_csv('allen_srl_event.csv')
df = df.iloc[4:13]
df



,trigger,v_s,v_e,arg0,a0_s,a0_e,arg1,a1_s,a1_e,arg2,a2_s,a2_e
4,were,100,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,were,121,121,NaN,NaN,NaN,NaN,0.0,1.0,exactly alike,123.0,124.0
6,stood,185,185,NaN,NaN,NaN,the tin soldiers,182.0,184.0,NaN,0.0,1.0
7,could,213,213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,surrounded,227,227,a number of little trees,222.0,226.0,"a piece of looking - glass , which was intended to represent a transparent lake",228.0,242.0,NaN,NaN,NaN
9,made,246,246,NaN,NaN,NaN,NaN,0.0,0.0,of wax,247.0,248.0
10,was,263,263,NaN,NaN,NaN,NaN,0.0,1.0,very pretty,264.0,265.0
11,was,323,323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,was,341,341,NaN,NaN,NaN,NaN,NaN,2.0,a dancer,342.0,343.0


In [3]:
#curate two_sentences pair, + the token position of the verb
sent_ls = []
event_ids = []
verbs = []

for i in range(1, df.shape[0]):
    concat = sent_df.iloc[i-1]["sentence"] + sent_df.iloc[i]["sentence"]
#     offset = int(sent_df.iloc[i-1]["start_id"])
#     l_event, r_event = int(df.iloc[i-1]["v_s"]) - offset, int(df.iloc[i]["v_s"]) - offset
    events = get_predicate(concat)
    tokens = events[0][-1]
    lverb, rverb = df.iloc[i-1]["trigger"], df.iloc[i]["trigger"]
    l_event = tokens.index(lverb)
    r_event = l_event + tokens[l_event+1:].index(rverb)+1
    
    sent_ls.append(concat)
    event_ids.append([l_event, r_event])
    verbs.append([lverb, rverb])
    print(concat)
    print(l_event, r_event)
    print(lverb,rverb)
    



NameError: name 'get_predicate' is not defined

In [49]:
examples = create_data_instances(sent_ls, event_ids)

print the triggers: ---------  

were were
print the triggers: ---------  

were stood
print the triggers: ---------  

stood could
print the triggers: ---------  

could surrounded
print the triggers: ---------  

surrounded made
print the triggers: ---------  

made was
print the triggers: ---------  

was was
print the triggers: ---------  

was was


In [54]:
with open('../data/matres/example.pickle', 'wb') as f:
    pickle.dump(examples, f)

# # Prepare Examples Dataset, save to Pickle format

In [4]:
from collections import OrderedDict
from utils import * 
from get_srl import *

04/20/2022 13:40:55 - INFO - allennlp.common.params -   id = coref-spanbert
04/20/2022 13:40:55 - INFO - allennlp.common.params -   registered_model_name = coref
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_class = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   registered_predictor_name = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   display_name = Coreference Resolution
04/20/2022 13:40:55 - INFO - allennlp.common.params -   task_id = coref
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_usage.archive_file = coref-spanbert-large-2020.02.27.tar.gz
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_usage.training_config = coref/coref_spanbert_large.jsonnet
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_usage.install_instructions = pip install allennlp==1.0.0 allennlp-models==1.0.0
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_usage.overrides = None
04/20/2022 13:40:55 - INFO - allennlp.

04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.paper.title = BART: Denosing Sequence-to-Sequence Pre-training for Natural Language Generation,Translation, and Comprehension
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.paper.url = https://api.semanticscholar.org/CorpusID:204960716
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.license = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.contact = allennlp-contact@allenai.org
04/20/2022 13:40:55 - INFO - allennlp.common.params -   intended_use.primary_uses = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   intended_use.primary_users = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   intended_use.out_of_scope_use_cases = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   factors.relevant_factors = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   factors.evaluation_factors = None
04/20/2022 13:40:55 -

04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_usage.overrides = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.description = The `MaskedLanguageModel` embeds some input tokens (including some which are masked), contextualizes them, then predicts targets for the masked tokens, computing a loss against known targets.
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.short_description = BERT-based masked language model
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.developed_by = Devlin et al
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.contributed_by = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.date = 2020-10-07
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.version = 1
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.model_type = BERT
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_detail

04/20/2022 13:40:55 - INFO - allennlp.common.params -   training_data.dataset.name = WebText corpus
04/20/2022 13:40:55 - INFO - allennlp.common.params -   training_data.dataset.url = https://github.com/openai/gpt-2-output-dataset
04/20/2022 13:40:55 - INFO - allennlp.common.params -   training_data.motivation = WebText emphasizes document quality. Only human-curated/filtered documents are scraped. Reddit outbound links which receive at least 3 karma points are taken as a proxy for human filtered webpages that are interesting.
04/20/2022 13:40:55 - INFO - allennlp.common.params -   training_data.preprocessing = Dragnet and [Newspaper](https://github.com/codelucas/newspaper) content extractors are used. Wikipedia articles are removed.
04/20/2022 13:40:55 - INFO - allennlp.common.params -   quantitative_analyses.unitary_results = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   quantitative_analyses.intersectional_results = None
04/20/2022 13:40:55 - INFO - allennlp.common.pa

04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.paper.title = RoBERTa: A Robustly Optimized BERT Pretraining Approach (Liu et al)
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.paper.url = https://api.semanticscholar.org/CorpusID:198953378
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.license = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   model_details.contact = allennlp-contact@allenai.org
04/20/2022 13:40:55 - INFO - allennlp.common.params -   intended_use.primary_uses = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   intended_use.primary_users = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   intended_use.out_of_scope_use_cases = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   factors.relevant_factors = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   factors.evaluation_factors = None
04/20/2022 13:40:55 - INFO - allennlp.common.params -   metrics.m

...

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.type = srl
04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.max_instances = None
04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.manual_distributed_sharding = False
04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.manual_multiprocess_sharding = False
04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.token_indexers = None
04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.domain_identifier = None
04/20/2022 13:41:07 - INFO - allennlp.common.params -   dataset_reader.bert_model_name = bert-base-uncased
04/20/2022 13:41:08 - INFO - allennlp.common.params -   type = from_instances
04/20/2022 13:41:08 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /tmp/tmp0w5zddyy/vocabulary.
04/20/2022 13:41:08 - INFO - allennlp.common.params -   model.type = srl_bert
04/20/2022 13:41:08 - INFO - allennlp.comm

04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.10.intermediate.dense.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.10.output.LayerNorm.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.10.output.LayerNorm.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.10.output.dense.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.10.output.dense.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.11.attention.output.LayerNorm.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.11.attention.output.LayerNorm.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.11.attention.output.dense.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.laye

04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.output.dense.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.self.key.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.self.key.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.self.query.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.self.query.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.self.value.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.attention.self.value.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.5.intermediate.dense.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.laye

04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.9.intermediate.dense.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.9.output.LayerNorm.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.9.output.LayerNorm.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.9.output.dense.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.encoder.layer.9.output.dense.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.pooler.dense.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      bert_model.pooler.dense.weight
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      tag_projection_layer.bias
04/20/2022 13:41:10 - INFO - allennlp.nn.initializers -      tag_projection_layer.weight
04/20/2022 13:41:10 - INFO - allennlp.models.archival -   removing temporary unarchived mode

In [5]:

sent = "I hated going to school; so, i will bomb my school!"


def parse(sent, event1_id, event2_id):
    od = OrderedDict()
    
    events = get_predicate(sent)
#     if not events:
#         continue
    tokens = events[0][-1]
    print("print the triggers: ---------  \n")
    print(tokens[event1_id], tokens[event2_id])
    tok_id = 0
    event1_span, event2_span = None, None
    for offset in range(len(sent)):
        if tok_id ==event1_id:
            event1_span = (offset, offset+len(tokens[tok_id])-1)
        elif tok_id ==event2_id:
            event2_span = (offset, offset+len(tokens[tok_id])-1)
        if tok_id >= len(tokens):
            break
        tok = tokens[tok_id]
        if sent[offset:offset+len(tok)]==tok:
            od['['+str(offset)+':'+str(offset+len(tok)) + ')'] = (tok, None)
            tok_id+=1
    return od, event1_span, event2_span



def create_data_instance(sent, event1_id, event2_id):
    """
    sentence is a string and event spans are a list of the spans of two events
    event1_id, event2_id are positional token id of the two event trigger tokens
    """ 
    doc_dict,e1_span,e2_span = parse(sent, event1_id, event2_id)
    left_event = Event(None, None, None, None, None, e1_span)
    right_event = Event(None, None, None, None, None, e2_span)
    v = dict()
    v['rel_type'],v['rev'],v['doc_dictionary'],v['event_labels'],v['doc_id'],v['left_event'],v['right_event'] = \
    'BEFORE', None, doc_dict, None, None, left_event, right_event
    return v


def create_data_instances(sents, event_ids):
    cnt = 0 
    d = dict()
    for sent, event_id, verb in zip(sents, event_ids, verbs):
        v = create_data_instance(sent, event_id[0], event_id[1])
        d['L_' + str(cnt)] = v
        cnt+=1
    return d
        
create_data_instance(sent,1,10)

# parse(sent)
# examples = create_data_instance(sent, event_spans)


print the triggers: ---------  

hated bomb


{'rel_type': 'BEFORE',
 'rev': None,
 'doc_dictionary': OrderedDict([('[0:1)', ('I', None)),
              ('[2:7)', ('hated', None)),
              ('[8:13)', ('going', None)),
              ('[14:16)', ('to', None)),
              ('[17:23)', ('school', None)),
              ('[23:24)', (';', None)),
              ('[25:27)', ('so', None)),
              ('[27:28)', (',', None)),
              ('[29:30)', ('i', None)),
              ('[31:35)', ('will', None)),
              ('[36:40)', ('bomb', None)),
              ('[41:43)', ('my', None)),
              ('[44:50)', ('school', None)),
              ('[50:51)', ('!', None))]),
 'event_labels': None,
 'doc_id': None,
 'left_event': <utils.Event at 0x7f2c9c4a2a00>,
 'right_event': <utils.Event at 0x7f2b391863d0>}

In [ ]:
examples = create_data_instances(sents, event_spans_ls)
examples

In [6]:
a = create_data_instance(sent,1,10)

print the triggers: ---------  

hated bomb


In [9]:
a['left_event'].span

(2, 6)